# Copy Number Pipeline

In [1]:
from __future__ import print_function
import os.path
import numpy as np
import pandas as pd
from IPython.display import Image,display
import dalmatian as dm
from IPython.core.display import HTML 
import sys
#loading package in another path
sys.path.insert(0, '..')
from src.CCLE_postp_function import *
from JKBio import TerraFunction as terra
# we will run some python functions directly in the notebook
import rpy2
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
# we will use google sheets (you need to have your storage and client files for your google account with an access to the required google sheets)- see sheeturl below
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have installed JKBio in the same folder as ccle_processing


## boot up

- you first need to go to [taiga](https://cds.team/taiga/dataset) and create some new datasets for the virtual release
- the easiest way to create a new dataset is to upload an empty file (since at least one file is required). This empty file can be deleted when you update the dataset with a new version

we are instanciating all the parameters needed for this pipeline to run

In [18]:
samplesetname = "20Q3"
prevname="20Q2"
prevversion=42
prevprevname ='20Q1'
prevprevversion= 40
virtual_public='public-20q3-3d35'
virtual_dmc='dmc-20q3-033d'
virtual_internal='internal-20q3-00d0'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_WES_CN_hg38"
source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
release = samplesetname

In [3]:
%%R
# same thing for R
release <- '20Q2'
prevname <- '20Q1'
version<-37
prevversion <- 40
prevprevversion<- 38
prevprevname <- '19Q4'
genome_version <- 'hg38'

In [ ]:
# we initialize the workspaces manager from dalmatian
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
# sometimes the bam files live under a different column in the workspace data
extract_to_change = {'from_arxspan_id': 'participant'}

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

## Adding new data

We are looking for new samples in a range of workspaces.

They are quite messy and might contains duplicates, contain broken file paths...

- We are thus looking at the bam files one by one and comparing them with our own bams. 
- We remove broken files, duplicates and add new version of a cell line's bam if we find some.

In [ ]:
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='wes', refurl=refsheet_url, wmfroms = [workspace1, workspace2, workspace3, workspace6], sources=[source1, source2, source3, source6], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

In [ ]:
newsample[1][newsample[1].index.isin(newsample[0])]

In [ ]:
#temporary renaming
rename = {'ACH-002446_2':'ACH-003000_1'}

In [ ]:
sample_ids, refsamples, ccle_name = createDatasetWithNewCellLines(refworkspace, samplesetname, wmfroms = [workspace1, workspace2, workspace3], sources=[source1, source2, source3], gsfolderto='gs://ccle_bams/wes/', match='ACH', participantslicepos=10, accept_unknowntypes=True, extract = extract_to_change, dry_run = False, rename=rename)

In [ ]:
refsamples[refsamples.index.isin(sample_ids)].WES_bam

## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the copy number dataset:

*   **BamToUnmappedRGBams_MC** vdauwera/BamToUnmappedRGBamsSnapshot ID: 3
*   **Generate_uBAM_File_List** gkugener/ArrayOfFilesToTxtSnapshot ID: 1
*   **Realign_WES_GATK4** gatk/PreProcessingForVariantDiscovery_GATK4Snapshot ID: 7
*   **CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9
*   **Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2

This output file for download will be saved under the sample set under the combined_seg_file attribute.

There are several other tasks in this workspace. In brief:

*   **CNV_Somatic_Panel_Workflow_Agilent_XX** gatk/CNV_Somatic_Panel_WorkflowSnapshot ID: 11. This task was used in this workspace to generate the Sanger PON. In the Sanger dataset, there is a set of 40 normal cell lines samples (cell lines derived from matched normal tissue). We can use these to generate a PON to normalize to rather than using the Agilent PON we use for the other CCLE cell lines. This leads to less noisy results. HOWEVER, results using the PON from this workflow should not use the X chromosome, as the sanger normals are not exclusively female or male (it is likely a mix).
*   **SANGER_PON_CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9. Same as the CNV_sample_XX_gatk, except that is uses the Sanger based PON. Should be used only for the Sanger cell lines.
*   **Sanger_PON_Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2. Aggregates the segment files for the samples that were run using the Sanger PON based CNV workflow.

## On Terra

In [ ]:
# a list of Terra workflows that are in the workspace and that we will call sequentially
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [ ]:
# see dalmatian
subid = refwm.create_submission(bamtoubam,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(ubamtofilelist,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(realign,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
# Testing out the XY PoN for CN characterization. Will test by producing an output in a different column from usual so it's easy to delete the column attribute later
# Also, need to make a split between Agilent and ICE samples..
samplesetname = '19Q4'
submission_id= refwm.create_submission("CNV_sample_XX",etype='sample_set',entity=samplesetname,expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

In [ ]:
submission_id= refwm.create_submission("CNV_sample_XX",etype='sample_set',entity=samplesetname,expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

### copy pairs data to sample data

In [ ]:
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[pairs.index.isin(tokeep)]
pairs = pairs[~pairs['called_copy_ratio_segments_tumor'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant_id'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

continuing

In [ ]:
submission_id = refwm.create_submission("Aggregate_CN_seg_files",entity=samplesetname)
terra.waitForSubmission(refworkspace,submission_id)

__we are getting the results file path__

In [ ]:
aggregated = refwm.get_entities('sample_set').loc[samplesetname]["combined_seg_file"]
aggregated

## On local

__We then save the workflow configurations used__

In [ ]:
terra.saveConfigs(refworkspace,'data/'+samplesetname+'/CNVconfig')

__delete unmapped bams generated during the process__

In [ ]:
toremove = ["readgroup_ubams",]
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
# sometimes the previous step does not work and you need to do it manually (you can run this to check it worked)
for val in samplesinset.readgroup_ubams:
    ubams = ''
    if not type(val) is list:
        continue 
    for v in val:
        ubams+=' '+v
    os.system('gsutil -m rm'+ubams)

__and move the hg38 aligned bams to our own datastorage bucket__

Note that we may encounter some WGS files, which need to go to a different folder from the WES bam files.

In [ ]:
onlycol = ['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index', 'hg38_analysis_ready_bam_md5']

wes_newgs = 'gs://cclebams/hg38_wes/'
wgs_newgs = 'gs://cclebams/hg38_wgs/'

wes_samples = samplesinset[samplesinset.datatype == 'wes']
wes_res = terra.changeGSlocation(refworkspace, newgs=wes_newgs, data=wes_samples, onlycol=onlycol, entity='samples', keeppath=False, dry_run = True)

wgs_samples = samplesinset[samplesinset.datatype == 'wgs']
wgs_res = terra.changeGSlocation(refworkspace, data=wgs_samples , newgs=wgs_newgs, onlycol=onlycol, entity='samples', keeppath=False, dry_run = True)

In [ ]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

__We download and reprocess removing the appended version and keeping only the newest versions__

In [ ]:
a = pd.read_csv("temp/cnv_ccle.called.seg", sep='\t')
a

In [ ]:
len(set(a.Sample))

### Get QC files

In [ ]:
dataBam = getWESQC(workspace=refworkspace ,only=[], qcname=["hg38_duplication_metrics","hg38_bqsr_report"])
dataCN = getWESQC(workspace=refworkspace ,only=[], qcname=["allelic_counts_tumor","delta_MAD_tumor","denoised_MAD_tumor","scaled_delta_MAD_tumor","denoised_copy_ratios_lim_4_plot_tumor","denoised_copy_ratios_plot_tumor","modeled_segments_plot_tumor"])

In [ ]:
ccle_refsamples = ccle_refsamples.set_index('cds_sample_id',drop=True)

In [ ]:
dataBam

In [ ]:
for k,v in dataCN.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'processing_qc'] = str(v)
for k,v in dataBam.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'bam_qc'] = str(v)
ccle_refsamples.to_csv('temp/newrefCN.csv')

### Priorization

add columns to seg file with arxspan ID, version. only keep the newest version for any given arxspan ID.
The process to keep the newest version of any given line is a little different from 20Q2 onwards, because don't have any dataset that uses the CDS-IDs for the data from 20Q1 or earlier.

We have to download the Taiga datasets from the previous quarter, see if we have any arxspan IDs with new data, and then replace with that data. We use the function called "removeOlderVersions" to do this.

In [ ]:
a_indexed = a.set_index('Sample')
a_indexed.index.names = ['sample_id']
a_indexed

In [ ]:
renaming = removeOlderVersions(names=a_indexed.index.tolist(), refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")

In [ ]:
a_indexed[a_indexed.index.isin(renaming.keys())].rename(renaming).to_csv("temp/cnv_ccle.called.seg", sep='\t')

#### saving samples used for 20Q2

In [ ]:
ccle_refsamples.loc[renaming.keys(),version]=1
ccle_refsamples.to_csv('temp/newrefCN.csv')

__If want to reprocess something__

In [ ]:
#%%R
#segments_unfiltered <- readr::read_csv('temp/wes.19Q3.segmented.cn')

## Get Absolute data from the mutation PiPeline and ...

## post Procesing

The post processing happens in R using guillaume's functions, in brief:

- processSegments
- filterForCCLE
- interpolateGapsInSegmented
- extendEndsOfSegments
- reprioritizeData

In [ ]:
%%R
source('src/load_libraries_and_annotations.R')
source("src/CCLE_postp_function.R")
library('celllinemapr')
library('magrittr')
library('taigr')
library('cdsomics')
library('readr')

In [ ]:
%%R
# Previous release copy number profiles. This line will need to be updated as well
wes.priority.cn.seg.profiles <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.file=paste0("wes.",prevname,".segmented"),data.version=prevversion) %>%
  dplyr::select(DepMap_ID, Chromosome, Start, End, Num_Probes, Segment_Mean, Source)

In [ ]:
%%R
print(mean(wes.priority.cn.seg.profiles$Segment_Mean))
print(max(wes.priority.cn.seg.profiles$Segment_Mean))
print(min(wes.priority.cn.seg.profiles$Segment_Mean))

### debugging interpolateGapsInSegmented(segments)$segs
**TODO:** getting error when call interpolateGapsInSegmented for the 20Q2 data. In particular, the mutate call at line 275 in the CCLE_postp_function.R file yields the error: "Error: must be a double vector, not an integer vector". The solution was changing `TRUE ~ as.integer(1))` to `TRUE ~ as.numeric(1))` in the interpolateGapsInSegmented function. I've pushed this to the Git repo.

In [ ]:
%%R
segments <- processSegments("temp/cnv_ccle.called.seg")
segments <- filterForCCLE(segments)
segments <- interpolateGapsInSegmented(segments)$segs
segments <- extendEndsOfSegments(segments,'../JKBio/data/hg38_cytoband.gz')
print(segments)
print(head(wes.priority.cn.seg.profiles))
# reprioritize also undo logtransform
print('all sources need to be named "Broad WES"')
print(wes.priority.cn.seg.profiles$Source)
segments_unfiltered <- reprioritizeData(segments, wes.priority.cn.seg.profiles)
# Fill in the gaps on the entire dataset
# Extend start sites to 1, end sites to the end of the chromosome?

In [ ]:
%%R
print(mean(segments_unfiltered$Segment_Mean))
print(max(segments_unfiltered$Segment_Mean))
print(min(segments_unfiltered$Segment_Mean))

In [ ]:
%%R
write.table(segments_unfiltered, file = paste0("temp/wes.",release,".segment.cn"), sep = ',', quote = F, row.names = F) 

### creating gene copy number

- generateEntrezGenes
- generateGeneLevelMatrixFromSegments

In [ ]:
%%R
entrezgenes <- generateEntrezGenes()

In [ ]:
%%R
head(corner(entrezgenes))

In [ ]:
%%R
res <- generateGeneLevelMatrixFromSegments(entrezgenes, segments_unfiltered)

In [ ]:
%%R
genematrix_unfiltered <- res$gene_level_data_hg38
corner(genematrix_unfiltered)

In [ ]:
%%R
print(mean(genematrix_unfiltered))
print(max(genematrix_unfiltered))

In [ ]:
%%R
# we save this table to be validated in python (quicker way to do it by passing it in cells)
write.table(genematrix_unfiltered, file = paste0('temp/wes.',release,'.gene.cn'), 
sep = ',', quote = F, row.names = T)

## Validation step

Once the files are saved, we load them back in python and do some validations, in brief:

- mean,max,var...
- to previous version: same mean,max,var...
- checkAmountOfSegments: flag any samples with a very high number of segments
- checkGeneChangeAccrossAll: flag any genes which stay at a similar value across all samples

In [66]:
genecn = pd.read_csv('temp/wes.'+release+'.gene.cn', sep = ',', index_col=0)

In [58]:
prevgenecn = tc.get(name='depmap-wes-cn-data-81a7', version=22, file='internal_20Q2_gene_cn')


[##################]100% |  33.6 MiB/s | 373.6 MiB / 373.6 MiB | Time:  0:00:11


In [65]:
set(prevgenecn.index) & set(blacklist)

{'ACH-002462',
 'ACH-002463',
 'ACH-002464',
 'ACH-002465',
 'ACH-002466',
 'ACH-002467',
 'ACH-002475'}

In [63]:
set(prevgenecn.index) - set(genecn.index)

set()

In [68]:
segmentcn = pd.read_csv('temp/wes.'+release+'.segment.cn', sep = ',')

In [70]:
# getting the previous versions to check that we have everything we should
prev = set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.'+prevname+'.gene', version=prevversion).index.tolist())
prevprev= set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.'+prevprevname+'.gene', version=prevprevversion).index.tolist())

	missing cell lines


In [22]:
new1 = set(genecn.index.values.tolist())
new2 = set(segmentcn['DepMap_ID'].values.tolist())
print(len(prev), len(prev & new1), len(new1), len(new1 & new2))

1760 1760 1767 1767


In [20]:
segmentcn[segmentcn.DepMap_ID=="ACH-001546"]

,DepMap_ID,Chromosome,Start,End,Num_Probes,Segment_Mean,Source


In [21]:
"ACH-001546" in segmentcn.index.tolist()

False

In [17]:
checkAmountOfSegments(segmentcn,thresh = 750)

ACH-000044 1202
ACH-001230 947
ACH-000836 1001
ACH-000216 925
ACH-000593 764
ACH-000090 1024
ACH-001955 1296
ACH-002204 1318
ACH-000327 819
ACH-000842 929
ACH-001045 822
ACH-000258 872
ACH-000659 1129
ACH-000118 794
ACH-000064 1203
ACH-000868 1422
ACH-001150 782
ACH-000941 813
ACH-000600 939
ACH-000870 1557
ACH-000068 812
ACH-000128 767
ACH-001101 1005
ACH-000028 868
ACH-000658 927
ACH-000690 771
ACH-000444 974
ACH-000167 838
ACH-001043 825
ACH-000901 816
ACH-000071 1287
ACH-001249 1756
ACH-001956 1368
ACH-000774 953
ACH-000419 826
ACH-000887 1408
ACH-000710 968
ACH-000578 869
ACH-001036 858
ACH-000550 974
ACH-000923 1469
ACH-001017 1223
ACH-001000 980
ACH-000300 1431
ACH-001079 2586
ACH-001234 819
ACH-001679 1227
ACH-001239 851
ACH-001094 1036
ACH-001225 792
ACH-002335 1312
ACH-000837 1015
ACH-000960 913
ACH-000452 816
ACH-000961 763
ACH-000635 1368
ACH-000848 1171
ACH-000356 1294
ACH-000454 1051
ACH-000458 762
ACH-001088 1337
ACH-001113 1072
ACH-001171 792
ACH-001071 1175
ACH-000854 

In [23]:
checkGeneChangeAccrossAll(genecn, thresh=1.5)

array([], dtype=object)

In [71]:
segmentcn.Start = segmentcn.Start.astype(int)
segmentcn.End = segmentcn.End.astype(int)

In [72]:
genecn.values.min(), genecn.values.mean(), genecn.values.max()

(2.780513339939832e-10, 1.0050513176124436, 7.220474335775503)

In [26]:
if(genecn.values.max() > 100):
    print("\n\n\nTOO HIGH, not LOG2 transformed!")
if(len(genecn.index.tolist()) > len(set(genecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
for k, val in samplesinset.iterrows():
    plot = val["modeled_segments_plot_tumor"]
    ! gsutil cp $plot temp/
    print(k)
    print(val['arxspan_id'])
    display(Image('temp/'+plot.split('/')[-1]))

These look bad in 20Q1: 
ACH-002511 (M140325), ACH-001370 (OCIP5X)

These CN plots subjectively appear to have too many segments in new 20Q2 samples: 
ACH-002399 (CDS-sukIAT, 21NT_1), ACH-002401 (CDS-tVy3GF, 21MT2_1), ACH-002400 (CDS-VUHMHG, 21MT1_1)

In [73]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_embargo = [i for i in gsheets['WES_embargo'].values.tolist() if str(i) != 'nan']
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if str(i) != 'nan']
blacklist = [i for i in gsheets['blacklist'].values.tolist() if str(i) != 'nan']

wes_embargo, wes_dmc_embargo, blacklist

(['ACH-001547',
  'ACH-001679',
  'ACH-001970',
  'ACH-001973',
  'ACH-002399',
  'ACH-002400',
  'ACH-002401',
  'ACH-002462',
  'ACH-002463',
  'ACH-002464',
  'ACH-002465',
  'ACH-002466',
  'ACH-002467'],
 [],
 ['ACH-001006',
  'ACH-001007',
  'ACH-001008',
  'ACH-001009',
  'ACH-001010',
  'ACH-001062',
  'ACH-001082',
  'ACH-001083',
  'ACH-001153',
  'ACH-001154',
  'ACH-001155',
  'ACH-001156',
  'ACH-001157',
  'ACH-001158',
  'ACH-001159',
  'ACH-001160',
  'ACH-001161',
  'ACH-001178',
  'ACH-001179',
  'ACH-001181',
  'ACH-001204',
  'ACH-001218',
  'ACH-001219',
  'ACH-001220',
  'ACH-001221',
  'ACH-001222',
  'ACH-001223',
  'ACH-001251',
  'ACH-001252',
  'ACH-001253',
  'ACH-001254',
  'ACH-001255',
  'ACH-001256',
  'ACH-001257',
  'ACH-001258',
  'ACH-001259',
  'ACH-001269',
  'ACH-001434',
  'ACH-001752',
  'ACH-001753',
  'ACH-001754',
  'ACH-001755',
  'ACH-001756',
  'ACH-001757',
  'ACH-001758',
  'ACH-001759',
  'ACH-001760',
  'ACH-001780',
  'ACH-001781',
  

# Upload to taiga

- we load the blacklisted/embargoed sample ids
- we log2 transform and create a file for each release (and one containing everything)
- we upload the files using taigapy in a corresponding taiga dataset with the corresponding description and also upload it to its virtual dataset

## we push full dataset version in depmap taiga CN

In [42]:
print(segmentcn.Segment_Mean.max(), segmentcn.Segment_Mean.mean(), segmentcn.Segment_Mean.min())
print(genecn.values.max(), genecn.values.mean(),genecn.values.min())

148.134924260474 1.068830521805282 1.92730498227434e-10
7.220474335775503 1.0050513176124436 2.780513339939832e-10


In [ ]:
## for genecn removing first blacklisted, then embargoed, to create two datasets
genecn = genecn.apply(lambda x: np.log2(1+x))
genecn.to_csv('temp/wes.'+release+'.gene.cn', index=True)
segmentcn.to_csv('temp/wes.'+release+'.segment.cn', index=False)

In [ ]:
if(genecn.values.max() > 100):
    print("\n\n\nTOO HIGH, not LOG2 transformed!")
if(len(genecn.index.tolist()) > len(set(genecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
print("Old genecn shape:", genecn.shape)
print("Old segmentcn shape:", segmentcn.shape)

In [ ]:
tc.update_dataset(dataset_permaname="segmented-cn-wes-prioritzed-7fe1", 
                  upload_file_path_dict={
                    'temp/wes.'+release+'.gene.cn': 'NumericMatrixCSV',
                    'temp/wes.'+release+'.segment.cn': 'TableCSV'},
                  changes_description=
"""
Removing duplication of ACH-000219 from segmentcn file to prevent issues in future releases, which use this Taiga dataset in the process of determining which new lines to release. Now both the genecn file and the segmentcn file have 1767 unique DepMap IDs.
""",
                  dataset_description=
"""
# Copy Number

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, log2(x+1) transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

These data are generated for Achilles to pull from to run CERES.

Versions to use:

v45 for 20Q2 (For Achilles QC, use v44 of segmentcn)
v40 for 20Q1
v38 for 19Q4
v33 for 19Q4
v25 for 19Q2 (hg38 aligned, Broad WES and Sanger WES based calls were generated from bam realignment. SNP based calls are still from liftover). The gene mapping script was updated to improve the gene level matrix (to remove NAs). The segment level matrix is untransformed relative CN. Gene level matrix is log2(CN + 1).
v20 for 19Q1 (version 21 is hg19)
v18: for 18Q4
v15: for 18Q3
v11: for 18Q2
Gene-level matrix in versions below 10 were using hg38 and not hg19. Version 11 is corrected and should be used instead

Calls on X, Y chromosome for profiles should not be used.

Prioritization is as follows:

Broad WES kept over everything
Sanger WES kept if:
This cell line did not fail fingerprinting
This cell line has no other CN data
This cell lines does not have CRISPR LFC data from the Achilles screen
This CN profile correlates better with Achilles CRISPR LFC data than Broad SNP CN OR the % gene-level difference between this cell lines CN profile from Sanger WES and Broad SNP < 2.5%
Broad SNP used for remaining lines with no Broad WES or with Sanger WES that does not pass the criteria above
The 'Source' column indicates which CN profile was used for that cell line.

version 6: renamed Sample column to CCLE_name for consistency for the Achilles pipeline

version 7: missing chordoma lines

version 8: fixed to names of two chordoma lines (changed suffix from CHORDOMA -> BONE) and removed renamed 
cell lines that were duplicated (with different names). Reran comparison using 18q2 LFC results. Gene level matrix will be generated for version 9

version 11: corrected error in gene-level matrix calculation (previously had been aligned to hg38 however alignment should be hg19). Segment level calls are unaffected.

versions 12-14: Sanger WES were multiplied by 2 so should not be used

version 15: internal segments and gene level matrices for 18q3 release including public version (removed black list lines and Broad WES < 6 months old). Gene level matrices are indexed using Broad IDs.

version 16: internal segments and gene level matrices for 18q4 release including public version (uses all SNP and only WES if those lines are present in the 18Q4 public Achilles dataset)

version 17: same as version 16 but with two additional line in the internal version

version 18: same as version 17 but switched one line in public to use SNP instead of WES because not in public Avana

version 20: two major changes occurred (1) we are using a FireCloud based pipeline for CN calling now for Broad WES data (2) we have moved to use hg38. This is accomplished by lifting over coordinates from hg19 to hg38 after processed by the CN pipeline.
v21 same as version 20, but we are using the original hg19 coordinates, not hg38

version 25: Broad WES and Sanger WES were realigned to hg38. SNP still uses liftover from hg19 to hg38

version 35: Seeing what went wrong with the upload.

version 36: problem with not log2 transforming the data

version 37: resolving the problem with log2 transforming the segment data

version 38: resolving the problem with log2 transforming the segment data

version 39: 20Q1. Samples ACH-002511 (M140325) and ACH-001370 (OCIP5X) appear to have too many segments looking at the CN profile.

version 40: unlog2 transforming segmentcn

version 41: 20Q2 (segmentcn is just relative copy number, whereas the genecn is log2(x+1) transformed). Added 7 new samples.
These CN plots subjectively appear to have too many segments in new 20Q2 samples: ACH-002399 (CDS-sukIAT, 21NT\_1), ACH-002401 (CDS-tVy3GF, 21MT2\_1), ACH-002400 (CDS-VUHMHG, 21MT1\_1)

version 42: **note: version 42 is missing some of the cell lines. Do not use** 

version 43: Resolving issue of no DepMap ID index in the genecn file. Duplicating the CN data in genecn and segmentcn for ACH-000219 so we have CN data for ACH-002874, the same cell line grown in different media. This step is required for Achilles / CERES.

version 44: Removing duplication of ACH-000219 from genecn file. The Achilles QC only needs the duplication in the segmentcn file. This change results in 1767 unique DepMap IDs in the genecn file, and 1768 unique DepMap IDs in the segmentcn file.

version 45: Removing duplication of ACH-000219 from segmentcn file to prevent issues in future releases, which use this Taiga dataset in the process of determining which lines should be released to Public. Now both the genecn file and the segmentcn file have 1767 unique DepMap IDs.

version 46: removing two weird undefined lines 

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean""")

## We push internal dataset with blacklisted removed

and we add it to eternal dataset and to virtual dataset

In [74]:
## for segment removing first blacklisted, then embargoed, to create two datasets
print(len(segmentcn))
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
print(len(segmentcn))
segmentcn.to_csv('temp/internal_'+release+'_segs_cn', index=False)
print(len(genecn))
genecn = genecn[~genecn.index.isin(blacklist)]
print(len(genecn))
genecn.to_csv('temp/internal_'+release+'_gene_cn', index=True)

341529
341053
1767
1765


In [ ]:
prevcn = tc.get(name='depmap-cn-data-81a7', version=24, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.DepMap_ID))
print("new lines")
newlines = set(genecn.DepMap_ID) - set(prevcn.DepMap_ID) 
newlines

In [75]:
tc.update_dataset(dataset_permaname="depmap-wes-cn-data-81a7", 
                  upload_file_path_dict={
                    'temp/internal_'+release+'_gene_cn': 'NumericMatrixCSV',
                    'temp/internal_'+release+'_segs_cn': 'TableCSV'},
                  dataset_description=
"""
# Copy Number


## ** Version 1 Internal 18Q1****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', 
                                  data.version=5, 
                                  data.file='gene_CN_WES_priority')
source_info <- data.frame(ccle_name=gsub("snp_|sangerWES_|ccleWES_|achillesWES_", 
                                         "", row.names(wes_pri)), 
                          source=gsub("_.*", "", row.names(wes_pri)))
wes_pri %<>% magrittr::set_rownames(source_info$ccle_name)

```

## ** Version 2 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=9, data.file='wes_priority_cn_gene_matrix') %>% log2()


```

## ** Version 3 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
## ** Version 4-6 Internal 18Q3****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Version 5 has updated cell line name mapping

Version 4 and 5 the segment level CN for Sanger's data is off by a factor of 2, version 6 corrects this

**** Version 7 Internal 18Q4****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

## ** Version 8-9 Internal 19Q1****

version 9 has the correct data for 19Q1

## ** Version 10-11 Internal 19Q2****

__version 11 added an additional 13 cell lines and adds the segment level copy number data__

## ** Version 12 Internal 19Q3****

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__. 


## ** Version 15 Internal 19Q4****

Adding 35 new cell lines

## ** Version 16 Internal 19Q4****
resolving problem with not having log2 transform 

## ** Version 17 Internal 19Q4****
resolving problem with having log2 transform on segments

## ** Version 18 Internal 20Q1****
adding 8 new cell lines

## ** Version 19 Internal 20Q1****
unlog2 transforming segmentcn

## ** Version 20 Internal 20Q1****
adding new cell lines

## ** Version 21 Internal 20Q1****
reparing some missing lines

## ** Version 22 Internal 20Q2****
adding new lines

## ** Version 23 Internal 20Q3****
nothing changed from 20Q2 updating the blacklist

## ** Version 24 Internal 20Q3****
updating the blacklists


Some cells lines have been flagged as:

 - having bad looking copy ration plots = ACH-002511 (M140325) and ACH-001370 (OCIP5X)
 - having too many segments (format: sample seg_count) = ACH-001079 2586, ACH-000044 1202, ACH-000258 872, ACH-001230 947, ACH-000068 812, ACH-000454 1051, ACH-000216 925, ACH-001150 782, ACH-001214 889, ACH-002335 1312, ACH-000836 1001, ACH-001957 1426, ACH-000960 913, ACH-000458 762, ACH-000578 869, ACH-000327 819, ACH-000090 1024, ACH-000488 954, ACH-000848 1171, ACH-000923 1469, ACH-000904 868, ACH-000452 816, ACH-000600 939, ACH-001656 902, ACH-000854 899, ACH-000774 953, ACH-001000 980, ACH-000941 813, ACH-000887 1408, ACH-001017 1223, ACH-001171 792, ACH-001071 1175, ACH-000593 764, ACH-001239 851, ACH-000071 1287, ACH-001956 1368, ACH-000509 873, ACH-002204 1318, ACH-000550 974, ACH-000738 1064, ACH-000870 1557, ACH-001036 858, ACH-001043 825, ACH-000028 868, ACH-001955 1296, ACH-000419 826, ACH-001234 819, ACH-001094 1036, ACH-001225 792, ACH-000118 794, ACH-000300 1431, ACH-001113 1072, ACH-001045 822, ACH-000444 974, ACH-000901 816, ACH-000865 1358, ACH-000961 763, ACH-001249 1756, ACH-000167 838, ACH-001101 1005, ACH-000842 929, ACH-000837 1015, ACH-000710 968, ACH-000195 2029, ACH-000064 1203, ACH-000690 771, ACH-000635 1368, ACH-000356 1294, ACH-000659 1129, ACH-000868 1422, ACH-000128 767, ACH-000658 927, ACH-001088 1337
 - Genes having a similar CN value accross all: []
 
## ** Version 20 Internal 20Q2****
Added 7 samples.

Some cells lines have been flagged as:

 - having bad looking copy ratio plots (appear to have too many segments): ACH-002399 (CDS-sukIAT, 21NT\_1), ACH-002401 (CDS-tVy3GF, 21MT2\_1), ACH-002400 (CDS-VUHMHG, 21MT1\_1)
 - having too many segments (format: sample seg_count): same as for 20Q1
 - Genes having a similar CN value accross all samples: []
 
 
## ** Version 21 Internal 20Q2****
 
Duplicating the CN data in genecn and segmentcn for ACH-000219 so we have CN data for ACH-002874, the same cell line grown in different media. This step is required for Achilles / CERES.

Version 22: removing two weird undefined lines 

## ** Version 23 Internal 20Q3****

same  as  20Q2 for this release. no new lines

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean

NEW LINES:
"""+newlines)

Uploading internal_20Q3_gene_cn...
hitting https://cds.team/taiga/api/datafile/cb5e4c48b1904f25ac1ccb8aef424184
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conve

'e31e5206fe674ae9aee8b2129cba7599'

In [76]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])

[('CCLE_gene_cn', 'depmap-wes-cn-data-81a7.24/internal_20Q3_gene_cn'), ('CCLE_segment_cn', 'depmap-wes-cn-data-81a7.24/internal_20Q3_segs_cn'), ('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-363a.25/internal_20Q3_counts'), ('CCLE_expression', 'depmap-rnaseq-expression-data-363a.25/internal_20Q3_proteincoding_tpm'), ('CCLE_mutations', 'depmap-mutation-calls-9be3.23/depmap_20q3_mutation_calls'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-363a.25/internal_20Q3_transcripts_tpm'), ('CCLE_expression_full', 'depmap-rnaseq-expression-data-363a.25/internal_20Q3_tpm')]
hitting https://cds.team/taiga/api/datafile/cda244e63bde446da7a1507d74f8a431
hitting https://cds.team/taiga/api/datafile/cda244e63bde446da7a1507d74f8a431
hitting https://cds.team/taiga/api/datafile/cda244e63bde446da7a1507d74f8a431
hitting https://cds.team/taiga/api/datafile/cda244e63bde446da7a1507d74f8a431
hitting https://cds.team/taiga/api/datafile/cda244e63bde446da7a1507d74f8a431
hitting https://cds.team/ta

## We add to dmc as in internal*

* **NOTE: change as of 20Q2 onwards**. We need to remove lines in WES_DMC_embargo from the Internal version of the CN datasets before we upload the `genecn` and `segmentcn` files to DMC.

In [77]:
## for segment removing first blacklisted, then embargoed, to create two datasets
print(len(segmentcn))
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(wes_dmc_embargo)]
print(len(segmentcn))
segmentcn.to_csv('temp/dmc_'+release+'_segs_cn', index=False)
print(len(genecn))
genecn = genecn[~genecn.index.isin(wes_dmc_embargo)]
print(len(genecn))
genecn.to_csv('temp/dmc_'+release+'_gene_cn', index=True)

341053
341053
1765
1765


In [ ]:
prevcn = tc.get(name='depmap-cn-data-9b9d', version=11, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.DepMap_ID))
print("new lines")
newlines = set(genecn.DepMap_ID) - set(prevcn.DepMap_ID) 
newlines

In [78]:
tc.update_dataset(dataset_permaname="depmap-cn-data-9b9d",
                upload_file_path_dict={
                    'temp/dmc_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/dmc_'+release+'_segs_cn': 'TableCSV',
                   },
                  changes_description=
"""
Adding 20Q2 samples.
""",
                
                  dataset_description="""
**** Version 1-2 DMC 19Q1****

version 2 contains the correct data for 19Q1

**** Version 3-4 DMC 19Q2****

__version 4 added an additional 13 cell lines and adds the segment level copy number data__

**** Version 5 DMC 19Q3***

**** Version 7 DMC 19Q4***
adding 35 new cell lines

**** Version 8 DMC 19Q4****
resolving problem with not having log2 transform 

**** Version 9 DMC 19Q4****
resolving problem with having log2 transformed the segments

**** Version 10 DMC 20Q1****
adding new samples

**** Version 11 DMC 20Q1****
unlog2 transforming segmentcn

**** Version 12 DMC 20Q2****
Adding samples to be included in 20Q2

**** Version 13 DMC 20Q2****
unknown changes

**** Version 14 DMC 20Q3****
updated blacklists

**** Version 15 DMC 20Q3****
issues with blacklists

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__.  The segment copy number data includes the mean segment copy number segments.

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean

NEW LINES:
"""+newlines)

Uploading dmc_20Q3_gene_cn...
hitting https://cds.team/taiga/api/datafile/dba99a9f1615439ea8e5f4ba3c39bc8c
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion

'72688a3cd3474a86ae553b11c1c03025'

In [79]:
AddToVirtual(virtual_dmc, "depmap-cn-data-9b9d", files=[('CCLE_gene_cn', 'dmc_'+release+'_gene_cn'),('CCLE_segment_cn', 'dmc_'+release+'_segs_cn')])

[('CCLE_gene_cn', 'depmap-cn-data-9b9d.15/dmc_20Q3_gene_cn'), ('CCLE_segment_cn', 'depmap-cn-data-9b9d.15/dmc_20Q3_segs_cn'), ('Achilles_gene_effect_unscaled', 'avana-consortium-20q3-902b.1/gene_effect_unscaled'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-80ef.15/dmc_20Q3_transcripts_tpm'), ('Achilles_raw_readcounts_failures', 'avana-consortium-20q3-902b.1/raw_readcounts_failing'), ('Achilles_gene_effect', 'avana-consortium-20q3-902b.1/gene_effect'), ('CCLE_mutations', 'depmap-mutation-calls-dfce.16/depmap_20q3_mutation_calls'), ('Achilles_gene_dependency', 'avana-consortium-20q3-902b.1/gene_dependency'), ('nonessentials', 'avana-consortium-20q3-902b.1/nonessential_genes'), ('Achilles_common_essentials', 'avana-consortium-20q3-902b.1/pan_dependent_genes'), ('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-80ef.15/dmc_20Q3_counts'), ('Achilles_dropped_guides', 'avana-consortium-20q3-902b.1/dropped_guides'), ('common_essentials', 'avana-consortium-20q3-902b.1/essentia

## We add to public as internal minus dmc embargoed and only cell lines from previous previous release (6 month)

In [80]:
print(len(segmentcn))
segmentcn = segmentcn[segmentcn.DepMap_ID.isin(prevprev)]
segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(set(wes_embargo))]
print(len(segmentcn))
segmentcn.to_csv('temp/public_' + release + '_segs_cn', index=False)
print(len(genecn))
genecn = genecn[genecn.index.isin(prevprev)]
genecn = genecn[~genecn.index.isin(set(wes_embargo))]
print(len(genecn))
genecn.to_csv('temp/public_'+release+'_gene_cn', index=True)

341053
337818
1765
1752


In [ ]:
prevcn = tc.get(name='depmap-cn-data-97cc', version=33, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prevcn.DepMap_ID) - set(genecn.DepMap_ID))
print("new lines")
newlines = set(genecn.DepMap_ID) - set(prevcn.DepMap_ID) 
newlines

In [81]:
tc.update_dataset(dataset_permaname='depmap-wes-cn-data-97cc',
                    upload_file_path_dict={
                    'temp/public_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/public_'+release+'_segs_cn': 'TableCSV',
                   },
                  changes_description=
"""
Adding 20Q2 samples.
""",
                  dataset_description="""
**** Versions 1-5 Public 18Q1****

Gene-level WES copy-number data for publicly accessible CCLE data. 

```

internal_lines <- readr::read_csv("~/Downloads/avana-broad-18q1_v2-sample-info.csv")$cell_line
public_lines <- readr::read_csv("~/Downloads/avana-public-tentative-18q1_v5-sample-info.csv")$cell_line
non_public_lines <- setdiff(internal_lines, public_lines)

full_cn_set <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', data.version=5, data.file='full_gene_CN')
source_info <- data.frame(source=gsub("_.*", "", row.names(full_cn_set)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(full_cn_set)),
                          row_idx=1:nrow(full_cn_set))
to_remove <- source_info %>%
  dplyr::filter(ccle_name %in% non_public_lines,
                source %in% c("ccleWES", "achillesWES"))
also_to_remove <- source_info %>%
                    dplyr::filter(source == "sangerWES")
indices_to_remove <- c(to_remove$row_idx, also_to_remove$row_idx) %>% unique()
indices_to_keep <- source_info %>%
  dplyr::filter(!(row_idx %in% indices_to_remove)) %>%
  dplyr::group_by(ccle_name) %>%
  dplyr::mutate(priority=ifelse(source == "snp", 4,
                                ifelse(source == "sangerWES", 3,
                                       ifelse(source == "ccleWES", 2, 1)))) %>%
  dplyr::filter(priority == min(priority)) %>%
  dplyr::ungroup()

public_cn <- full_cn_set[indices_to_keep$row_idx,]
source_info <- data.frame(source=gsub("_.*", "", row.names(public_cn)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(public_cn)))
public_cn %<>% magrittr::set_rownames(source_info$ccle_name)
```

CN data are on a log2 scale.

`WES_source_info` tracks the source data for each cell line. Sources are `snp`, `achillesWES`, `ccleWES`, and `sangerWES`

NOTE: Version 1 contained WES data from cell lines not available in the 18Q1 Public release. Versions 2-4 contained Sanger's WES CN data

**** Version 6 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=10, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

**** Version 7 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```


**** Version 8-9 Public 18Q3****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```
Version 8 the segment level CN for Sanger's data is off by a factor of 2, version 9 corrects this

includes cell lines that should not be public

**** Version 10 Public 18Q1, 18Q2, 18Q3****

__use version 10 for 18Q1, 18Q2 and 18Q3 datasets__ 

Version 10 is the most up-to-date version of "public\_18Q3\_gene\_cn.csv". The three datasets have been updated to remove cell lines that should not have been made public. They are named in the portal and google bucket for portal downloads as v2, e.g. public\_18Q3\_gene\_cn\_v2.csv.

__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 11 Public 18Q4****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 12-14 Public 19Q1****

version 14 contains the correct data for 19Q1

version 13 is the same as v12 except that it uses the original hg19 coordinates not hg38. The Achilles public data set uses the hg19 coordinates. 

**** Version 15-16 Public 19Q2****

__version 16 also adds the segment level copy number data__

**** Version 17-18 Public 19Q3****


**** Version 23 Public 19Q4****
adding new cell lines

**** Version 24 Internal 19Q4****
resolving problem with not having log2 transform 

**** Version 25 Internal 19Q4****
another issue in log transform

**** Version 26 Internal 19Q4****
unlog2 transforming segmentcn FINAL

**** Version 27 Internal 20Q1****
adding new samples

**** Version 28 Internal 20Q1****
log 2 transform issues

**** Version 29 Internal 20Q1****
readding one missing column in segments

**** Version 30 Internal 20Q2****
Adding new samples

**** Version 31 Internal 20Q2****
unknown changes

**** Version 32 Internal 20Q3****
same  as  20Q2 for this release. no new lines

**** Version 33 Internal 20Q3****
updated blacklist

## Gene level CN data:

__data is hg38 liftover__

__Description__: log2 + 1 gene level copy number data (data is log2 transformed with a __pseudocount of 1__ added). It uses hg19 coordinates. Also the segment level copy number data.

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean, CCLE\_name

NEW LINES:
"""+newlines)

Uploading public_20Q3_gene_cn...
hitting https://cds.team/taiga/api/datafile/bcbe83ee64c94aba90c814a849d73146
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Convers

'1516e31ba2da4c8b93b5d221a0533026'

In [82]:
# To add to a virtual dataset
AddToVirtual(virtual_public, "depmap-wes-cn-data-97cc", files=[('CCLE_gene_cn', 'public_'+release+'_gene_cn'),('CCLE_segment_cn', 'public_'+release+'_segs_cn')])

[('CCLE_gene_cn', 'depmap-wes-cn-data-97cc.33/public_20Q3_gene_cn'), ('CCLE_segment_cn', 'depmap-wes-cn-data-97cc.33/public_20Q3_segs_cn'), ('CCLE_expression', 'depmap-rnaseq-expression-data-ccd0.22/public_20Q3_proteincoding_tpm'), ('CCLE_expression_full', 'depmap-rnaseq-expression-data-ccd0.22/public_20Q3_tpm'), ('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-ccd0.22/public_20Q3_counts'), ('CCLE_mutations', 'depmap-mutation-calls-9a1a.19/depmap_20q3_mutation_calls'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-ccd0.22/public_20Q3_transcripts_tpm')]
hitting https://cds.team/taiga/api/datafile/77c75977a31d42f68527c72b84e2f609
hitting https://cds.team/taiga/api/datafile/77c75977a31d42f68527c72b84e2f609
hitting https://cds.team/taiga/api/datafile/77c75977a31d42f68527c72b84e2f609
hitting https://cds.team/taiga/api/datafile/77c75977a31d42f68527c72b84e2f609
hitting https://cds.team/taiga/api/datafile/77c75977a31d42f68527c72b84e2f609
hitting https://cds.team/taiga/api/data